In [1]:
from llama_index.llms.gemini import Gemini
img_model = Gemini(model="models/gemini-2.0-flash-exp-image-generation", temperature=1)


In [9]:
from gtts import gTTS 
import os 

myText = "sample text"
language = 'en'

output = gTTS(text=myText, lang=language, slow=False)

output.save("output.mp3")
os.system("start output.mp3")

sh: 1: start: not found


32512